In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print("berhasil mengimport yang diperlukan")

berhasil mengimport yang diperlukan


In [10]:
# Dataset utama
data = pd.read_excel('2000_Kalimat_Kinerja_Pesantren_Unik.xlsx')  # pastikan kolom: Kalimat, Aspek, Pesantren

# Lexicon
positive_words = set(pd.read_csv('positive.tsv', header=None)[0].tolist())
negative_words = set(pd.read_csv('negative.tsv', header=None)[0].tolist())

print(data.head())
print(data.shape)

   No                                            Kalimat                Aspek  \
0   1  Pengajar memiliki kompetensi memberikan perhat...        Kualitas Guru   
1   2  Pengalaman guru masih perlu ditingkatkan. Meng...        Kualitas Guru   
2   3  Beberapa guru masih perlu ditingkatkan. Mengur...        Kualitas Guru   
3   4  Prestasi santri meraih prestasi akademik dan n...             Prestasi   
4   5  Proses belajar mengajar sangat efektif dan men...  Kualitas Pengajaran   

  Sentimen             Pesantren  
0  Positif  As-Salafiyah Parappe  
1  Negatif    Syekh Hasan Yamani  
2  Negatif  As-Salafiyah Parappe  
3  Positif            Al-Ikhlash  
4  Positif            Al-Ikhlash  
(324, 5)


In [19]:
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|@\w+|#[A-Za-z0-9_]+", "", text)
    text = re.sub(f"[{string.punctuation}]", " ", text)
    text = stemmer.stem(text)
    return text

data['Clean'] = data['Kalimat'].apply(preprocess)

print(data[['Kalimat', 'Clean']].head(10)) 



                                             Kalimat  \
0  Pengajar memiliki kompetensi memberikan perhat...   
1  Pengalaman guru masih perlu ditingkatkan. Meng...   
2  Beberapa guru masih perlu ditingkatkan. Mengur...   
3  Prestasi santri meraih prestasi akademik dan n...   
4  Proses belajar mengajar sangat efektif dan men...   
5  Santri kurang termotivasi untuk berprestasi. P...   
6  Prestasi santri banyak penghargaan tingkat reg...   
7  Materi pelajaran terasa membosankan dan monoto...   
8  Proses belajar mengajar sangat jelas dan inter...   
9  Pengajaran kadang kurang sabar saat mengajar. ...   

                                               Clean  
0  ajar milik kompetensi beri perhati penuh santr...  
1  alam guru masih perlu tingkat kurang kualitas ...  
2  beberapa guru masih perlu tingkat kurang kuali...  
3  prestasi santri raih prestasi akademik dan non...  
4  proses ajar ajar sangat efektif dan tarik ajar...  
5  santri kurang motivasi untuk prestasi perlu up... 

In [ ]:
def get_sentiment(text):
    print(text)
    pos, neg = 0, 0
    for word in text.split():
        if word in positive_words:
            pos += 1
        elif word in negative_words:
            neg += 1
    if pos > neg:
        return 2  # Positif
    elif neg > pos:
        return 0  # Negatif
    else:
        return 1  # Netral

data['Sentimen'] = data['Clean'].apply(get_sentiment)
print(data[['Kalimat', 'Clean', 'Sentimen']].head(10)) 
print("Jumlah data per kelas sebelum balancing:")
print(data['Sentimen'].value_counts())

ajar milik kompetensi beri perhati penuh santri rasa dukung
alam guru masih perlu tingkat kurang kualitas ajar
beberapa guru masih perlu tingkat kurang kualitas ajar
prestasi santri raih prestasi akademik dan non akademik dapat perhati dari masyarakat
proses ajar ajar sangat efektif dan tarik ajar langsung senang
santri kurang motivasi untuk prestasi perlu upaya baik segera
prestasi santri banyak harga tingkat regional tingkat reputasi pesantren
materi ajar asa bosan dan monoton butuh baik segera
proses ajar ajar sangat jelas dan interaktif guru sangat dedikasi
ajar kadang kurang sabar saat ajar butuh latih tambah
kurang dukung untuk ikut lomba prestasi harus tingkat dengan latih
guru guru sangat ramah dan bantu kepada santri
metode ajar sangat struktur dan mudah paham ajar langsung senang
beberapa fasilitas asa sempit dan kurang nyaman perlu perhati lebih
ajar milik kompetensi ramah dan bantu kepada santri
ajar sudah rutin tanpa banyak inovasi santri ikut dengan biasa saja
santri kura

In [15]:
# Menyeimbangkan Data dengan Oversampling

df_0 = data[data['Sentimen'] == 0]
df_1 = data[data['Sentimen'] == 1]
df_2 = data[data['Sentimen'] == 2]

max_count = max(len(df_0), len(df_1), len(df_2))

df_0 = resample(df_0, replace=True, n_samples=max_count, random_state=42)
df_1 = resample(df_1, replace=True, n_samples=max_count, random_state=42)
df_2 = resample(df_2, replace=True, n_samples=max_count, random_state=42)

data_balanced = pd.concat([df_0, df_1, df_2])
print("Jumlah data per kelas setelah balancing:")
print(data_balanced['Sentimen'].value_counts())

InvalidParameterError: The 'n_samples' parameter of resample must be an int in the range [1, inf) or None. Got 0 instead.

In [ ]:
# Tokenisasi dan Padding

tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(data_balanced['Clean'])
sequences = tokenizer.texts_to_sequences(data_balanced['Clean'])
X = pad_sequences(sequences, maxlen=100)
y = data_balanced['Sentimen'].values

In [ ]:
#  Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

NameError: name 'X' is not defined

In [ ]:
# Bangun Model LSTM
model = Sequential([
    Embedding(input_dim=5000, output_dim=100, input_length=100),
    SpatialDropout1D(0.2),
    LSTM(198, dropout=0.2),
    Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Training Model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


In [ ]:
# Evaluasi Model
# Prediksi
y_pred = model.predict(X_test).argmax(axis=1)

# Laporan Klasifikasi
print(classification_report(y_test, y_pred, target_names=['Negatif', 'Netral', 'Positif']))

# Confusion Matrix
print(confusion_matrix(y_test, y_pred))


In [ ]:
# Visualisasi Akurasi
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Akurasi Training dan Validasi")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
